In [1]:
from sklearn.model_selection import StratifiedKFold, train_test_split
import pandas as pd
pd.set_option('display.max_columns',100)
import os
import seaborn as sns
sns.set()
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
plt.rcParams['figure.figsize'] = 8, 5
plt.style.use("fivethirtyeight")
for dirname, _, filenames in os.walk('Predicting_Coupon_Redemption'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
train_df = pd.read_csv('../Data/train.csv')
campaign_data_df = pd.read_csv('../Data/campaign_data.csv')
coupon_item_mapping_df = pd.read_csv('../Data/coupon_item_mapping.csv')
customer_demographics_df = pd.read_csv('../Data/customer_demographics.csv')
customer_transaction_data_df = pd.read_csv('../Data/customer_transaction_data.csv')
item_data_df = pd.read_csv('../Data/item_data.csv')
test_df = pd.read_csv('../Data/test.csv')
sub_df = pd.read_csv('../Data/sample_submission.csv')




In [3]:
# Feature Engineering in campaign_data.csv data.
campaign_data_df['start_date'] = pd.to_datetime(campaign_data_df['start_date'], format='%d/%m/%y', dayfirst=True)
campaign_data_df['end_date'] = pd.to_datetime(campaign_data_df['end_date'], format='%d/%m/%y', dayfirst=True)
campaign_data_df['diff_d'] = (campaign_data_df['end_date'] - campaign_data_df['start_date']) / np.timedelta64(1,'D')
campaign_data_df.drop(['start_date','end_date'], axis=1, inplace=True)

No Feature Engineering required for the train.csv data.

No Feature Engineering required for the item_data.csv data.

No Feature Engineering required for the coupon_item_mapping.csv data.

Feature Engineering in customer_demographics.csv data.

In [4]:
lb = LabelEncoder()
customer_demographics_df['age_range'] = lb.fit_transform(customer_demographics_df['age_range'])
print(customer_demographics_df['age_range'])

0      5
1      3
2      1
3      1
4      3
      ..
755    2
756    3
757    3
758    1
759    1
Name: age_range, Length: 760, dtype: int32


In [5]:
# customer_transaction_data.csv
customer_transaction_data_df['date'] = pd.to_datetime(customer_transaction_data_df['date'], format='%Y-%m-%d')
customer_transaction_data_df['date_d'] = customer_transaction_data_df['date'].dt.day.astype('category')
customer_transaction_data_df['date_m'] = customer_transaction_data_df['date'].dt.month.astype('category')
customer_transaction_data_df['date_w'] = customer_transaction_data_df['date'].dt.week.astype('category')
customer_transaction_data_df.drop(['date'], axis=1, inplace=True)

In [6]:
tgroup = customer_transaction_data_df.groupby(['customer_id']).sum().reset_index()

In [7]:
# Merge Data.
train_campaign_data = pd.merge(train_df,campaign_data_df,on='campaign_id',how="left")
test_campaign_data= pd.merge(test_df,campaign_data_df,on='campaign_id',how="left")

In [8]:
# Coupon Item Mapping.
coupon_item_mapping_item_data = pd.merge(coupon_item_mapping_df, item_data_df, on='item_id', how="left")
mci_group = pd.DataFrame()
mci_group[['coupon_id','no_of_items']] = coupon_item_mapping_item_data.groupby('coupon_id').count().reset_index()[
    ['coupon_id','item_id']]
mci_group[['brand_type','category']] = coupon_item_mapping_item_data.groupby('coupon_id').max().reset_index()[
    ['brand_type','category']]


In [9]:
mdtg = pd.merge(tgroup,customer_demographics_df,on='customer_id',how='outer')


In [10]:
# Merge all.
mergeddata = pd.merge(train_campaign_data, mdtg, on=['customer_id'], how='left')
mergeddata = pd.merge(mergeddata, mci_group, on=['coupon_id'], how='left')

In [11]:
mergeddata2 = pd.merge(test_campaign_data, mdtg, on=['customer_id'], how='left')
mergeddata2 = pd.merge(mergeddata2, mci_group, on=['coupon_id'], how='left')
id_df = mergeddata2['id']
# mergeddata.drop(['id'],axis=1,inplace=True)
# mergeddata2.drop(['id'],axis=1,inplace=True)


In [12]:
print(mergeddata['redemption_status'].value_counts())
print(mergeddata['redemption_status'].value_counts(normalize=True))

0    77640
1      729
Name: redemption_status, dtype: int64
0    0.990698
1    0.009302
Name: redemption_status, dtype: float64


In [13]:
# Checking Missing values.
print(mergeddata.isnull().sum())
percent_missing = mergeddata.isnull().sum() * 100 / len(mergeddata)
missing_value_df = pd.DataFrame({'column_name': mergeddata.columns,
                                 'percent_missing': percent_missing})
print(missing_value_df)
mergeddata.drop(['no_of_children','age_range','marital_status','rented','family_size','income_bracket'], axis=1, inplace=True)
print(mergeddata.isnull().sum())

id                       0
campaign_id              0
coupon_id                0
customer_id              0
redemption_status        0
campaign_type            0
diff_d                   0
item_id                  0
quantity                 0
selling_price            0
other_discount           0
coupon_discount          0
age_range            34708
marital_status       52975
rented               34708
family_size          34708
no_of_children       64313
income_bracket       34708
no_of_items              0
brand_type               0
category                 0
dtype: int64
                         column_name  percent_missing
id                                id         0.000000
campaign_id              campaign_id         0.000000
coupon_id                  coupon_id         0.000000
customer_id              customer_id         0.000000
redemption_status  redemption_status         0.000000
campaign_type          campaign_type         0.000000
diff_d                        diff_d      

In [14]:
# Checking Missing values.
print(mergeddata2.isnull().sum())
percent_missing = mergeddata2.isnull().sum() * 100 / len(mergeddata2)
missing_value_df = pd.DataFrame({'column_name': mergeddata2.columns,
                                 'percent_missing': percent_missing})
print(missing_value_df)
mergeddata2.drop(['no_of_children','age_range','marital_status','rented','family_size','income_bracket'], axis=1, inplace=True)
print(mergeddata2.isnull().sum())

id                     0
campaign_id            0
coupon_id              0
customer_id            0
campaign_type          0
diff_d                 0
item_id                0
quantity               0
selling_price          0
other_discount         0
coupon_discount        0
age_range          19287
marital_status     32144
rented             19287
family_size        19287
no_of_children     40064
income_bracket     19287
no_of_items            0
brand_type             0
category               0
dtype: int64
                     column_name  percent_missing
id                            id         0.000000
campaign_id          campaign_id         0.000000
coupon_id              coupon_id         0.000000
customer_id          customer_id         0.000000
campaign_type      campaign_type         0.000000
diff_d                    diff_d         0.000000
item_id                  item_id         0.000000
quantity                quantity         0.000000
selling_price      selling_price     

In [15]:
# Feature Encoding.
cols = ['campaign_type','brand_type','category']
lb = LabelEncoder()
for i in cols:
    mergeddata[i] = lb.fit_transform(mergeddata[i])

In [16]:
for i in cols:
    mergeddata2[i] = lb.fit_transform(mergeddata2[i])


In [17]:
X = mergeddata.drop(['redemption_status'],axis=1)
Y = mergeddata['redemption_status']

In [18]:
# Handling class imbalance.
# Technique - 1.
fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=2020)
i = 1
auc = []

for train_index, test_index in fold.split(X, Y):
    x_train, x_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = Y.iloc[train_index], Y.iloc[test_index]
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_val = scaler.transform(x_val)
    mergeddata2 = scaler.transform(mergeddata2)
    m = LGBMClassifier(random_state=80) # 93.23
    m.fit(x_train, y_train)
    pred_prob1 = m.predict_proba(x_val)
    auc.append(roc_auc_score(y_val, pred_prob1[:, 1]))
    i = i + 1

print("AUC Score")
print(sum(auc)/10)

AUC Score
0.927883427257051


In [19]:
# Technique-2:SMOTE
# Oversample the dataset.

from imblearn.over_sampling import SMOTE
from collections import Counter
oversample = SMOTE()
X,Y = oversample.fit_resample(X,Y)
counter = Counter(Y)
print(counter)

Counter({0: 77640, 1: 77640})


In [20]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state = 101)

In [21]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [22]:
classifier_o = AdaBoostClassifier(random_state=20)  # 98.11
classifier_o.fit(X_train,y_train)
pred_prob1 = classifier_o.predict_proba(X_test)
print(roc_auc_score(y_test, pred_prob1[:, 1]))

0.9817829370816472


In [23]:
classifier_o = GradientBoostingClassifier(random_state=30)  # 98.74
classifier_o.fit(X_train,y_train)
pred_prob1 = classifier_o.predict_proba(X_test)
print(roc_auc_score(y_test, pred_prob1[:, 1]))

0.9879257666718586


In [24]:
classifier_o = LGBMClassifier(random_state=10)  # 99.78
classifier_o.fit(X_train,y_train)
pred_prob1 = classifier_o.predict_proba(X_test)
print(roc_auc_score(y_test,pred_prob1[:,1]))

0.9981286001204357


In [25]:
# LGBM Hyperparameter Tuning using Grid Search.
from sklearn.model_selection import GridSearchCV
param_grid = {'learning_rate': [0.1,0.5],
                 'max_depth': [4,5,6],
                 'num_leaves': [10,20],
                 'feature_fraction': [0.6,0.8],
                 'subsample': [0.2,0.6],
                 'objective': ['binary'],
              'metric': ['auc'],
              'is_unbalance':[False],
              'boosting':['gbdt'],
              'num_boost_round':[100],
              'early_stopping_rounds':[30]}

# Build and fit the GridSearchCV.
grid = GridSearchCV(estimator=classifier_o, param_grid=param_grid,
                    cv=10,verbose=10)
grid_results = grid.fit(X_train,y_train,eval_set = (X_test,y_test))
# Summarize the results in a readable format.
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))

Fitting 10 folds for each of 48 candidates, totalling 480 fits
[CV 1/10; 1/48] START boosting=gbdt, early_stopping_rounds=30, feature_fraction=0.6, is_unbalance=False, learning_rate=0.1, max_depth=4, metric=auc, num_boost_round=100, num_leaves=10, objective=binary, subsample=0.2
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] early_stopping_round is set=30, early_stopping_rounds=30 will be ignored. Current value: early_stopping_round=30
[LightGBM] [Warning] num_iterations is set=100, num_boost_round=100 will be ignored. Current value: num_iterations=100
[1]	valid_0's auc: 0.896488
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.952437
[3]	valid_0's auc: 0.956725
[4]	valid_0's auc: 0.958693
[5]	valid_0's auc: 0.957703
[6]	valid_0's auc: 0.959822
[7]	va

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] early_stopping_round is set=30, early_stopping_rounds=30 will be ignored. Current value: early_stopping_round=30
[LightGBM] [Warning] num_iterations is set=100, num_boost_round=100 will be ignored. Current value: num_iterations=100
[1]	valid_0's auc: 0.896494
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.952479
[3]	valid_0's auc: 0.957065
[4]	valid_0's auc: 0.958589
[5]	valid_0's auc: 0.960454
[6]	valid_0's auc: 0.961406
[7]	valid_0's auc: 0.96165
[8]	valid_0's auc: 0.963671
[9]	valid_0's auc: 0.963773
[10]	valid_0's auc: 0.964707
[11]	valid_0's auc: 0.964776
[12]	valid_0's auc: 0.965029
[13]	valid_0's auc: 0.964746
[14]	valid_0's auc: 0.965858
[15]	valid_0's auc: 0.966415
[16]	valid_0's auc: 0.96768
[

[1]	valid_0's auc: 0.896512
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.952426
[3]	valid_0's auc: 0.956964
[4]	valid_0's auc: 0.958784
[5]	valid_0's auc: 0.957969
[6]	valid_0's auc: 0.959641
[7]	valid_0's auc: 0.959957
[8]	valid_0's auc: 0.96387
[9]	valid_0's auc: 0.964083
[10]	valid_0's auc: 0.964964
[11]	valid_0's auc: 0.96528
[12]	valid_0's auc: 0.965618
[13]	valid_0's auc: 0.964895
[14]	valid_0's auc: 0.965371
[15]	valid_0's auc: 0.965627
[16]	valid_0's auc: 0.966887
[17]	valid_0's auc: 0.968205
[18]	valid_0's auc: 0.96963
[19]	valid_0's auc: 0.969791
[20]	valid_0's auc: 0.970532
[21]	valid_0's auc: 0.970498
[22]	valid_0's auc: 0.971344
[23]	valid_0's auc: 0.971765
[24]	valid_0's auc: 0.972158
[25]	valid_0's auc: 0.972803
[26]	valid_0's auc: 0.973308
[27]	valid_0's auc: 0.974143
[28]	valid_0's auc: 0.974569
[29]	valid_0's auc: 0.974764
[30]	valid_0's auc: 0.97493
[31]	valid_0's auc: 0.974936
[32]	valid_0's auc: 0.9755
[33]	valid_0's auc: 0.9765

[36]	valid_0's auc: 0.977417
[37]	valid_0's auc: 0.97759
[38]	valid_0's auc: 0.977993
[39]	valid_0's auc: 0.978883
[40]	valid_0's auc: 0.979058
[41]	valid_0's auc: 0.979585
[42]	valid_0's auc: 0.979663
[43]	valid_0's auc: 0.980273
[44]	valid_0's auc: 0.980489
[45]	valid_0's auc: 0.980732
[46]	valid_0's auc: 0.981021
[47]	valid_0's auc: 0.981629
[48]	valid_0's auc: 0.981896
[49]	valid_0's auc: 0.982173
[50]	valid_0's auc: 0.982449
[51]	valid_0's auc: 0.982633
[52]	valid_0's auc: 0.982973
[53]	valid_0's auc: 0.983152
[54]	valid_0's auc: 0.983281
[55]	valid_0's auc: 0.983409
[56]	valid_0's auc: 0.983583
[57]	valid_0's auc: 0.983802
[58]	valid_0's auc: 0.98394
[59]	valid_0's auc: 0.984122
[60]	valid_0's auc: 0.98467
[61]	valid_0's auc: 0.984884
[62]	valid_0's auc: 0.985217
[63]	valid_0's auc: 0.985742
[64]	valid_0's auc: 0.985869
[65]	valid_0's auc: 0.985936
[66]	valid_0's auc: 0.986308
[67]	valid_0's auc: 0.986412
[68]	valid_0's auc: 0.986572
[69]	valid_0's auc: 0.986777
[70]	valid_0's au

[70]	valid_0's auc: 0.98708
[71]	valid_0's auc: 0.987455
[72]	valid_0's auc: 0.987746
[73]	valid_0's auc: 0.988028
[74]	valid_0's auc: 0.988137
[75]	valid_0's auc: 0.988319
[76]	valid_0's auc: 0.988472
[77]	valid_0's auc: 0.988551
[78]	valid_0's auc: 0.988732
[79]	valid_0's auc: 0.988825
[80]	valid_0's auc: 0.988877
[81]	valid_0's auc: 0.989081
[82]	valid_0's auc: 0.989183
[83]	valid_0's auc: 0.989212
[84]	valid_0's auc: 0.989281
[85]	valid_0's auc: 0.989457
[86]	valid_0's auc: 0.989525
[87]	valid_0's auc: 0.989634
[88]	valid_0's auc: 0.989702
[89]	valid_0's auc: 0.989767
[90]	valid_0's auc: 0.989807
[91]	valid_0's auc: 0.989858
[92]	valid_0's auc: 0.989903
[93]	valid_0's auc: 0.989978
[94]	valid_0's auc: 0.990033
[95]	valid_0's auc: 0.99007
[96]	valid_0's auc: 0.990238
[97]	valid_0's auc: 0.990324
[98]	valid_0's auc: 0.990417
[99]	valid_0's auc: 0.990491
[100]	valid_0's auc: 0.990544
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.990544
[CV 9/10; 1/48] END boos

...

[1]	valid_0's auc: 0.942577
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.961724
[3]	valid_0's auc: 0.966723
[4]	valid_0's auc: 0.973125
[5]	valid_0's auc: 0.976812
[6]	valid_0's auc: 0.979775
[7]	valid_0's auc: 0.98315
[8]	valid_0's auc: 0.985622
[9]	valid_0's auc: 0.986993
[10]	valid_0's auc: 0.987623
[11]	valid_0's auc: 0.989244
[12]	valid_0's auc: 0.990187
[13]	valid_0's auc: 0.990729
[14]	valid_0's auc: 0.991797
[15]	valid_0's auc: 0.992378
[16]	valid_0's auc: 0.992863
[17]	valid_0's auc: 0.99347
[18]	valid_0's auc: 0.994124
[19]	valid_0's auc: 0.994515
[20]	valid_0's auc: 0.994836
[21]	valid_0's auc: 0.995225
[22]	valid_0's auc: 0.995652
[23]	valid_0's auc: 0.995944
[24]	valid_0's auc: 0.996169
[25]	valid_0's auc: 0.996222
[26]	valid_0's auc: 0.996482
[27]	valid_0's auc: 0.996584
[28]	valid_0's auc: 0.996807
[29]	valid_0's auc: 0.996945
[30]	valid_0's auc: 0.997053
[31]	valid_0's auc: 0.997154
[32]	valid_0's auc: 0.997274
[33]	valid_0's auc: 0.

[26]	valid_0's auc: 0.995968
[27]	valid_0's auc: 0.99616
[28]	valid_0's auc: 0.996353
[29]	valid_0's auc: 0.9965
[30]	valid_0's auc: 0.996591
[31]	valid_0's auc: 0.99676
[32]	valid_0's auc: 0.996915
[33]	valid_0's auc: 0.997012
[34]	valid_0's auc: 0.99717
[35]	valid_0's auc: 0.997304
[36]	valid_0's auc: 0.997384
[37]	valid_0's auc: 0.997389
[38]	valid_0's auc: 0.997494
[39]	valid_0's auc: 0.997603
[40]	valid_0's auc: 0.997642
[41]	valid_0's auc: 0.997718
[42]	valid_0's auc: 0.997807
[43]	valid_0's auc: 0.997844
[44]	valid_0's auc: 0.997889
[45]	valid_0's auc: 0.998014
[46]	valid_0's auc: 0.99813
[47]	valid_0's auc: 0.998172
[48]	valid_0's auc: 0.998222
[49]	valid_0's auc: 0.998273
[50]	valid_0's auc: 0.998321
[51]	valid_0's auc: 0.998379
[52]	valid_0's auc: 0.998427
[53]	valid_0's auc: 0.998467
[54]	valid_0's auc: 0.998521
[55]	valid_0's auc: 0.998645
[56]	valid_0's auc: 0.998684
[57]	valid_0's auc: 0.998702
[58]	valid_0's auc: 0.998727
[59]	valid_0's auc: 0.998764
[60]	valid_0's auc: 

[52]	valid_0's auc: 0.998683
[53]	valid_0's auc: 0.998733
[54]	valid_0's auc: 0.998795
[55]	valid_0's auc: 0.998717
[56]	valid_0's auc: 0.998729
[57]	valid_0's auc: 0.998741
[58]	valid_0's auc: 0.998782
[59]	valid_0's auc: 0.998811
[60]	valid_0's auc: 0.998822
[61]	valid_0's auc: 0.998877
[62]	valid_0's auc: 0.998896
[63]	valid_0's auc: 0.998918
[64]	valid_0's auc: 0.998916
[65]	valid_0's auc: 0.998955
[66]	valid_0's auc: 0.998971
[67]	valid_0's auc: 0.998991
[68]	valid_0's auc: 0.99902
[69]	valid_0's auc: 0.998323
[70]	valid_0's auc: 0.998785
[71]	valid_0's auc: 0.998792
[72]	valid_0's auc: 0.998807
[73]	valid_0's auc: 0.998819
[74]	valid_0's auc: 0.998697
[75]	valid_0's auc: 0.998823
[76]	valid_0's auc: 0.998833
[77]	valid_0's auc: 0.998846
[78]	valid_0's auc: 0.998856
[79]	valid_0's auc: 0.998856
[80]	valid_0's auc: 0.99888
[81]	valid_0's auc: 0.998919
[82]	valid_0's auc: 0.998929
[83]	valid_0's auc: 0.998939
[84]	valid_0's auc: 0.998902
[85]	valid_0's auc: 0.998423
[86]	valid_0's a

[77]	valid_0's auc: 0.998726
[78]	valid_0's auc: 0.998718
[79]	valid_0's auc: 0.998747
[80]	valid_0's auc: 0.998541
[81]	valid_0's auc: 0.998703
[82]	valid_0's auc: 0.998738
[83]	valid_0's auc: 0.99879
[84]	valid_0's auc: 0.998814
[85]	valid_0's auc: 0.998824
[86]	valid_0's auc: 0.998824
[87]	valid_0's auc: 0.998831
[88]	valid_0's auc: 0.998848
[89]	valid_0's auc: 0.996994
[90]	valid_0's auc: 0.998747
[91]	valid_0's auc: 0.998764
[92]	valid_0's auc: 0.998804
[93]	valid_0's auc: 0.998821
[94]	valid_0's auc: 0.998826
[95]	valid_0's auc: 0.998834
[96]	valid_0's auc: 0.998846
[97]	valid_0's auc: 0.998864
[98]	valid_0's auc: 0.998876
[99]	valid_0's auc: 0.998877
[100]	valid_0's auc: 0.998884
Did not meet early stopping. Best iteration is:
[71]	valid_0's auc: 0.999003
[CV 8/10; 48/48] END boosting=gbdt, early_stopping_rounds=30, feature_fraction=0.8, is_unbalance=False, learning_rate=0.5, max_depth=6, metric=auc, num_boost_round=100, num_leaves=20, objective=binary, subsample=0.6; total time

[CV 10/10; 48/48] END boosting=gbdt, early_stopping_rounds=30, feature_fraction=0.8, is_unbalance=False, learning_rate=0.5, max_depth=6, metric=auc, num_boost_round=100, num_leaves=20, objective=binary, subsample=0.6; total time=   0.9s
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] early_stopping_round is set=30, early_stopping_rounds=30 will be ignored. Current value: early_stopping_round=30
[LightGBM] [Warning] num_iterations is set=100, num_boost_round=100 will be ignored. Current value: num_iterations=100
[1]	valid_0's auc: 0.936361
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.960008
[3]	valid_0's auc: 0.964246
[4]	valid_0's auc: 0.969141
[5]	valid_0's auc: 0.974862
[6]	valid_0's auc: 0.979691
[7]	valid_0's auc: 0.982227
[8]	valid_0's auc: 0.

In [26]:
classifier_o = LGBMClassifier(random_state=10,boosting='gbdt',feature_fraction=0.8,is_unbalance=False,learning_rate=0.5,max_depth=6,metric='auc',num_boost_round=100,num_leaves=20,objective='binary',subsample=0.2)  # 99.78
classifier_o.fit(X_train,y_train)
pred_prob1 = classifier_o.predict_proba(X_test)
print(roc_auc_score(y_test,pred_prob1[:,1]))  # 99.89

0.9989905466389208


In [27]:
pred = classifier_o.predict(mergeddata2)
sub_df['redemption_status'] = pred
sub_df.to_csv('final_predictions.csv', index=False)